In [73]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [60]:
#Re-size image 
image_size = [224, 224]

In [61]:
train_path = 'Dataset/train'
test_path = 'Dataset/test'

In [63]:
vgg = VGG16(input_shape=image_size + [3], include_top = False)
#Cutting the last layer

In [26]:
#Don't need train this model
for layer in vgg.layers:
    layer.trainable = False

In [27]:
folders = glob('/Users/vignaya/Documents/python/Machine_Learning_A-Z_(Codes_and_Datasets)/Part 8 - Deep Learning/Section 40 - Convolutional Neural Networks (CNN)/Python/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set/*')

In [28]:
x = Flatten()(vgg.output)
predictions = Dense(1, activation = 'sigmoid')(x)

In [29]:
model = Model(inputs = vgg.input, outputs = predictions)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [30]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

##### Ignore this

In [34]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
                        '/Users/vignaya/Documents/python/Machine_Learning_A-Z_(Codes_and_Datasets)/Part 8 - Deep Learning/Section 40 - Convolutional Neural Networks (CNN)/Python/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set/',
                                                 target_size = image_size,
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [35]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
    '/Users/vignaya/Documents/python/Machine_Learning_A-Z_(Codes_and_Datasets)/Part 8 - Deep Learning/Section 40 - Convolutional Neural Networks (CNN)/Python/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set/',
                                            target_size = image_size,
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [39]:
test_set.image_shape

(224, 224, 3)

In [65]:
model.fit(training_set, epochs = 2, steps_per_epoch=len(training_set), validation_data=testing_set, validation_steps=len(testing_set))

Epoch 1/2
250/250 [==============================] - 3757s 15s/step - loss: 0.2747 - accuracy: 0.8801 - val_loss: 0.1822 - val_accuracy: 0.9260
Epoch 2/2
250/250 [==============================] - 4647s 19s/step - loss: 0.1945 - accuracy: 0.9176 - val_loss: 0.1729 - val_accuracy: 0.9340


In [68]:
pd.DataFrame(model.history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.274741,0.880125,0.182215,0.926
1,0.194523,0.917625,0.172934,0.934


In [69]:
pred = model.predict(testing_set)

In [72]:
pred = pred>0.5

In [76]:
testing_set.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [77]:
confusion_matrix(testing_set.classes, pred)

array([[479, 521],
       [501, 499]])

In [78]:
print(classification_report(testing_set.classes, pred))

              precision    recall  f1-score   support

           0       0.49      0.48      0.48      1000
           1       0.49      0.50      0.49      1000

    accuracy                           0.49      2000
   macro avg       0.49      0.49      0.49      2000
weighted avg       0.49      0.49      0.49      2000

